In [5]:
#批量合并nc并做提取
import pandas as pd
import os
import csv
import numpy as np
import xarray as xr
from operator import itemgetter


def extr(path):
    ls = os.listdir(path + '\\nc_files')
    
    #读取站点经纬度
    loc = pd.read_csv(path + "\\fluxnet.csv",sep=',',header = None)
    loc = loc.values
    #设置站点经纬度构成的数组
    latitude = loc[1:,1]         
    longitude = loc[1:,2]        
    all_sites = []
    for item in loc[1:,0]:
        all_sites.append(item)
    os.mkdir(path + '\\sites')
    os.chdir(path + '\\sites')
    for site in all_sites:
        os.mkdir(site)
    
    #读取并写入csv
    for date in ls:
        for i in range(267):
            with xr.load_dataset(path + '\\nc_files\\'+ date).sel(lat = latitude[i], lon = longitude[i], method= 'nearest') as test:         
                test_all = test.to_dataframe()       
                os.chdir(path + '\\sites\\'+all_sites[i])
                test_all.to_csv(date +'.csv')       #输出csv
    
    
    loc = pd.read_csv(path + "\\fluxnet.csv",sep=',',header = None)
    loc = loc.values
    #设置站点经纬度构成的数组
    latitude = loc[1:,1]         
    longitude = loc[1:,2]        
    all_sites = []
    for item in loc[1:,0]:
        all_sites.append(item)
    print(len(all_sites))

    os.mkdir(path + '\\all_flux_sites')
    j = 1
    header = np.loadtxt(path + '\\sites\\' + all_sites[0] + '\\' + ls[0] + '.csv', dtype = 'str', delimiter = ',')
    
    #表格合并&逐站点输出总表
    for site in all_sites:
        total = header[0]
        for dirs,folders,files in os.walk(path + '\\sites\\' + site):
            for file in files:
                total = np.vstack((total, pd.read_csv(path + '\\sites\\' + site+ '\\' + file).values))
            #print(total)
            total = sorted(total[1:], key = itemgetter(0))       #按时间顺序排列总表
            output = np.vstack((header[0], total))
            os.mkdir(path + '\\all_flux_sites\\' + site)
            output = pd.DataFrame(output)
            output.to_csv(path + '\\all_flux_sites\\' + site + '\\Total.csv',index = False, header = None)
            
    print("finished")

#调用主函数
extr('D:\\2012')

267
finished
